# Linear Programming

<img src="figures/ilp.png" width=650>

## Introduction

Discrete optimization is a branch of optimization methodology which deals with discrete quantities i.e. non-continuous functions. It is quite ubiquitous in diverse applications such as financial investment, diet planning, manufacturing processes, and player or schedule selection for professional sports.

Linear and (mixed) integer programming are techniques to solve problems which can be formulated within the framework of discrete optimization.

Knowledge of such optimization techniques is extremely useful for data scientists and machine learning (ML) practitioners as discrete and continuous optimization lie at the heart of modern ML and AI systems as well as data-driven business analytics processes.

There is a long and rich history of the theoretical development of robust and efficient solvers for optimization problems. However, focusing on practical applications, we will skip that history and move straight to the part of learning how to use programmatic tools to formulate and solve such optimization problems.

There are many excellent optimization packages in Python. In this tutorial, we will specifically talk about PuLP. But before going to the Python library, let us get a sense of the kind of problem we can solve with it.

## An example problem

Suppose you are in charge of the diet plan for high school lunch. Your job is to make sure that the students get the right balance of nutrition from the chosen food.

However, there are some restrictions in terms of budget and the variety of food that needs to be in the diet to make it interesting. The following table shows, in detail, the complete nutritional value for each food item, and their maximum/minimum daily intake.

<img src="figures/food.png" width=650>

The discrete optimization problem is simple: Minimize the cost of the lunch given these constraints (on total calories but also on each of the nutritional component e.g. cholesterol, vitamin A, calcium, etc.

Essentially, in a casual mathematical language, the problem is,

$ Minimize: \sum c_i x_i $

$ s.t. calories_{lower} < \sum calories_i * x_i < calories_{upper} $

$ s.t. protine_{lower} < \sum protine_i * x_i < protine_{upper} $

Notice that the inequality relations are all linear in nature i.e. the variables $x_i$ are multiplied by constant coefficients and the resulting terms are bounded by constant limits and that’s what makes this problem solvable by an LP technique.

You can imagine that this kind of problem may pop up in business strategy extremely frequently. Instead of nutritional values, you will have profits and other types of business yields, and in place of price/serving, you may have project costs in thousands of dollars. As a manager, your job will be to choose the projects, that give maximum return on investment without exceeding a total budget of funding the project.

Similar optimization problem may crop up in a factory production plan too, where maximum production capacity will be functions of the machines used and individual products will have various profit characteristics. As a production engineer, your job could be to assign machine and labor resources carefully to maximize the profit while satisfying all the capacity constraints.

Fundamentally, the commonality between these problems from disparate domains is that they involve maximizing or minimizing a linear objective function, subject to a set of linear inequality or equality constraints.

For the diet problem, the objective function is the total cost which we are trying to minimize. The inequality constraints are given by the minimum and maximum bounds on each of the nutritional components.

There are many libraries in the Python ecosystem for this kind of optimization problems. PuLP (Python library for linear optimization) is an open-source linear programming (LP) package which largely uses Python syntax and comes packaged with many industry-standard solvers. It also integrates nicely with a range of open source and commercial LP solvers.

We begin with the standard imports:

In [ ]:
#!pip install pulp

import pandas as pd
from pulp import *

### Read the given nutrition dataset into a Pandas DataFrame object
Note we are reading only the first 64 rows with `nrows=64` argument because we just want to read all the nutrients informtion and not the maximum/minimum bounds in the dataset. We will enter those bounds in the optimization problem separately.

In [ ]:
df = pd.read_excel("data/diet - medium.xls",nrows=17)

### Show the dataset

In [ ]:
df

### Create the `PuLP` problem variable. Since it is a cost minimization problem, we need to use `LpMinimize`

In [ ]:
# Create the 'prob' variable to contain the problem data
prob = LpProblem("Simple Diet Problem",LpMinimize)

### Create a list of food items from the dataset

In [ ]:
# Creates a list of the Ingredients
food_items = list(df['Foods'])

In [ ]:
print("So, the food items to consdier, are\n"+"-"*100)
for f in food_items:
    print(f,end=', ')

### Create a dictinary of costs for all food items

In [ ]:
costs = dict(zip(food_items,df['Price/Serving']))

In [ ]:
costs

### Create a dictionary of calories for all food items

In [ ]:
calories = dict(zip(food_items,df['Calories']))

### Create a dictionary of cholesterol for all food items

In [ ]:
cholesterol = dict(zip(food_items,df['Cholesterol (mg)']))

### Create a dictionary of total fat for all food items

In [ ]:
fat = dict(zip(food_items,df['Total_Fat (g)']))

### Create a dictionary of sodium for all food items

In [ ]:
sodium = dict(zip(food_items,df['Sodium (mg)']))

### Create a dictionary of carbohydrates for all food items

In [ ]:
carbs = dict(zip(food_items,df['Carbohydrates (g)']))

### Create a dictionary of dietary fiber for all food items

In [ ]:
fiber = dict(zip(food_items,df['Dietary_Fiber (g)']))

### Create a dictionary of protein for all food items

In [ ]:
protein = dict(zip(food_items,df['Protein (g)']))

### Create a dictionary of vitamin A for all food items

In [ ]:
vit_A = dict(zip(food_items,df['Vit_A (IU)']))

### Create a dictionary of vitamin C for all food items

In [ ]:
vit_C = dict(zip(food_items,df['Vit_C (IU)']))

### Create a dictionary of calcium for all food items

In [ ]:
calcium = dict(zip(food_items,df['Calcium (mg)']))

### Create a dictionary of iron for all food items

In [ ]:
iron = dict(zip(food_items,df['Iron (mg)']))

### Create a dictionary of food items with lower bound

In [ ]:
# A dictionary called 'food_vars' is created to contain the referenced Variables
food_vars = LpVariable.dicts("Food",food_items,0,cat='Continuous')

In [ ]:
food_vars

### Adding the objective function to the problem

In [ ]:
# The objective function is added to 'prob' first
prob += lpSum([costs[i]*food_vars[i] for i in food_items]), "Total Cost of the balanced diet"

### Adding the calorie constraints to the problem

In [ ]:
prob += lpSum([calories[f] * food_vars[f] for f in food_items]) >= 800.0, "CalorieMinimum"
prob += lpSum([calories[f] * food_vars[f] for f in food_items]) <= 1300.0, "CalorieMaximum"

### Adding other nutrient constraints to the problem one by one...

Exercise: 
Change the following ``markdown`` cells (cholesterol to iron) to ``code`` cells and comment on the changes to the results.

### Cholesterol
prob += lpSum([cholesterol[f] * food_vars[f] for f in food_items]) >= 30.0, "CholesterolMinimum"
prob += lpSum([cholesterol[f] * food_vars[f] for f in food_items]) <= 240.0, "CholesterolMaximum"

### Fat
prob += lpSum([fat[f] * food_vars[f] for f in food_items]) >= 40.0, "FatMinimum"
prob += lpSum([fat[f] * food_vars[f] for f in food_items]) <= 100.0, "FatMaximum"

### Sodium
prob += lpSum([sodium[f] * food_vars[f] for f in food_items]) >= 500.0, "SodiumMinimum"
prob += lpSum([sodium[f] * food_vars[f] for f in food_items]) <= 2000.0, "SodiumMaximum"

### Carbs
prob += lpSum([carbs[f] * food_vars[f] for f in food_items]) >= 130.0, "CarbsMinimum"
prob += lpSum([carbs[f] * food_vars[f] for f in food_items]) <= 450.0, "CarbsMaximum"

### Fiber
prob += lpSum([fiber[f] * food_vars[f] for f in food_items]) >= 125.0, "FiberMinimum"
prob += lpSum([fiber[f] * food_vars[f] for f in food_items]) <= 250.0, "FiberMaximum"

### Protein
prob += lpSum([protein[f] * food_vars[f] for f in food_items]) >= 60.0, "ProteinMinimum"
prob += lpSum([protein[f] * food_vars[f] for f in food_items]) <= 100.0, "ProteinMaximum"

### Vitamin A
prob += lpSum([vit_A[f] * food_vars[f] for f in food_items]) >= 1000.0, "VitaminAMinimum"
prob += lpSum([vit_A[f] * food_vars[f] for f in food_items]) <= 10000.0, "VitaminAMaximum"

### Vitamin C
prob += lpSum([vit_C[f] * food_vars[f] for f in food_items]) >= 400.0, "VitaminCMinimum"
prob += lpSum([vit_C[f] * food_vars[f] for f in food_items]) <= 5000.0, "VitaminCMaximum"

### Calcium
prob += lpSum([calcium[f] * food_vars[f] for f in food_items]) >= 300.0, "CalciumMinimum"
prob += lpSum([calcium[f] * food_vars[f] for f in food_items]) <= 1500.0, "CalciumMaximum"

### Iron
prob += lpSum([iron[f] * food_vars[f] for f in food_items]) >= 10.0, "IronMinimum"
prob += lpSum([iron[f] * food_vars[f] for f in food_items]) <= 40.0, "IronMaximum"

In [ ]:
# Fat
prob += lpSum([fat[f] * food_vars[f] for f in food_items]) >= 20.0, "FatMinimum"
prob += lpSum([fat[f] * food_vars[f] for f in food_items]) <= 50.0, "FatMaximum"

# Carbs
prob += lpSum([carbs[f] * food_vars[f] for f in food_items]) >= 130.0, "CarbsMinimum"
prob += lpSum([carbs[f] * food_vars[f] for f in food_items]) <= 200.0, "CarbsMaximum"

# Fiber
prob += lpSum([fiber[f] * food_vars[f] for f in food_items]) >= 60.0, "FiberMinimum"
prob += lpSum([fiber[f] * food_vars[f] for f in food_items]) <= 125.0, "FiberMaximum"

# Protein
prob += lpSum([protein[f] * food_vars[f] for f in food_items]) >= 100.0, "ProteinMinimum"
prob += lpSum([protein[f] * food_vars[f] for f in food_items]) <= 150.0, "ProteinMaximum"

### Writing problem data to a `.lp` file

In [ ]:
# The problem data is written to an .lp file
prob.writeLP("SimpleDietProblem.lp")

### Run the solver

In [ ]:
# The problem is solved using PuLP's choice of Solver
prob.solve()

### Print the problem solution status `'optimal'`, `'infeasible'`, `'unbounded'` etc...

In [ ]:
# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])

### Scan through the problem variables and print out only if the variable quanity is positive i.e. it is included in the optimal solution

In [ ]:
print("Therefore, the optimal (least cost) balanced diet consists of\n"+"-"*110)
for v in prob.variables():
    if v.varValue>0:
        print(v.name, "=", v.varValue)

### Print the optimal diet cost

In [ ]:
print("The total cost of this balanced diet is: ${}".format(round(value(prob.objective),2)))